In [1]:
import os
import json
import re 

from collections import defaultdict
from konlpy.tag import Mecab

In [2]:
mecab = Mecab()

In [11]:
nbest_name = 'nbest_predictions.json'
nbest_paths = [
    '/opt/ml/dy',
    '/opt/ml/ys'
]


In [12]:
candidates = defaultdict(lambda: defaultdict(int))

for path in nbest_paths:
    with open(os.path.join(path, nbest_name), 'r') as f:
        nbest_preds = json.load(f)

    for mrc_id, preds in nbest_preds.items():
        for p in preds:
            text = p['text']  # .strip()

            # TODO: add weights for each file
            prob = p['probability']
            candidates[mrc_id][text] += prob

In [13]:
blacklist = ['최소', '대략', '한때', '간혹', '최소한', '필수', "이따금","아마","절대로","무조건","한때","대략","오직", "오로지","감히","최소","아예","반드시","꼭","때때로","이미", "심지어" ,"종종","졸곧","약간","기꺼이", "비록","꾸준히","일부러","어쩔", "문득", "어쨌든", "순전히", "필수","자칫", "다소", "간혹", "적어도", "왜냐하면", "아무래도"]

def valid_answer(text):
    # This function only tests for answers with Korean
    if not re.search('[가-힣]', text):
        return True
    elif text.strip() in blacklist:
        return False
        
    part_of_speech = [x[1][0] for x in mecab.pos(text)]

    # TODO: M tag 수식언만 나오는 경우? 
    if len(part_of_speech) == 1 and 'M' in part_of_speech:
        return False
    return True

    # return 'N' in part_of_speech

In [14]:
voted_preds = {}
pattern = re.compile('[.\",\{\}\[\]\(\)\<\>]')

for mrc_id, aggregated_preds in candidates.items():
    p = [(text, sum_probs) for text, sum_probs in aggregated_preds.items()]
    sorted_scores = sorted(p, key=lambda x: x[1], reverse=True)

    selected = 0
    pred = sorted_scores[selected][0]

    while (re.sub(pattern, '', pred) == '' or len(pred) > 30 or not valid_answer(pred)) and selected < len(sorted_scores)-1:
        selected += 1
        pred = sorted_scores[selected][0]

    voted_preds[mrc_id] = pred

In [15]:
output_dir = '/opt/ml/output'

with open(os.path.join(output_dir, 'ensemble_predictions.json'), 'w+') as f:
    json.dump(voted_preds, f, ensure_ascii=False, indent = 4)